# Intro to `pandas` — Exercises

We continue exploring the Pandas package for simple data handling tasks using geoscience data examples. 

## Reading and writing files

Pandas reads files from disk or the web — [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) is a list of all the 20 or so formats that it can read and write. A very common one is Excel's `.xlsx` format, so let's load one!

Note that this requires `openpyxl` to be installed, eg with `uv add openpyxl` or `pip install openpxyl`.

The data is the same as used in this study: http://www.kgs.ku.edu/PRS/publication/2003/ofr2003-30/index.html

From that poster:

> The Panoma Field (2.9 TCF gas) produces from Permian Council Grove Group marine carbonates and nonmarine silicilastics in the Hugoton embayment of the Anadarko Basin. It and the Hugoton Field, which has produced from the Chase Group since 1928, the top of which is 300 feet shallower have combined to produce 27 TCF gas, making it the largest gas producing area in North America. Both fields are stratigraphic traps with their updip west and northwest limits nearly coincident. Maximum recoveries in the Panoma are attained west of center of the field. Deeper production includes oil and gas from Pennsylvanian Lansing-Kansas City, Marmaton, and Morrow and the Mississippian.

For Excel files, we can load specific sheets by passing the `sheet_name` argument:

In [ ]:
import pandas as pd

url = "https://github.com/scienxlab/datasets/raw/refs/heads/main/kgs/panoma_data.xlsx"

df = pd.read_excel(url, sheet_name='data')
df.head()

Without it, we get the first sheet, which in this case is not the data that we want, but it may still be useful:

In [ ]:
pd.read_excel(url)

We can also load all the worksheets as a dictionary by passing `sheet_name=None`, which gives us a dictionary of DataFrames, with the key being the sheet name:

In [ ]:
pd.read_excel(url, sheet_name=None)

Other formats are usually loaded in a similarly way, using the `pd.read_*` pattern: `pd.read_csv`, `pd.read_csv` and so on.

## Column types

### EXERCISE

Load the "data" sheet again, but use the "columns" sheet to set the types of the columns.

You may need to think about:

- Renaming one or more columns in the "columns" sheet.
- Constructing a dictionary like `{'<column_name>: <type_as_string>'}`
- Using that dictionary when loading the "data" sheet.

## Replacing `Facies`

Facies uses an encoding, but there are more natural descriptions in the "facies" sheet.

Make a new column called "Lithology" and populate it with the descriptions corresponding to the facies codes. Use an appropriate dtype.

Remove the `-` from `"Non-marine sandstone" in the descriptions. 

Finally, drop the "Facies" column.

In [ ]:
df = df.drop(columns=['Facies'])
df.head()

## Locations

Sometimes it's convenient to have 'group' data in the main DataFrame. For example, we could add columns from "wells" to `df`.

Add the 3 columns. Make sure to use the appropriate types, eg timestamp for "Completion date".

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">


<h3>Exercise</h3>

Using the `df` object, constructed from the sheet called "data", do the following:

* Make a new dataframe with only the `Well Name`, `Depth` and `GR` columns.
* How many rows are there in the `LUKE G U` well?
* In the `CROSS H CATTLE` well, what is the mean of `GR` when depth > 850 m?
</div>

In [ ]:
grouped = df.loc[df['Depth'] > 850].groupby(df['Well Name'])
grouped['GR'].mean()

## 

## Writing data out

Pandas can write to about 20 different formats. Writing data out is similarly simple to reading it in, using one of the range of `to_*` functions.

In this case, we will go with a simple `csv` format, which [most people love](https://github.com/medialab/xan/blob/master/docs/LOVE_LETTER.md):

In [ ]:
df.to_csv('./data/Panoma_Field_Permian.csv', index=False)

Pickle, Feather, or Parquet files are more performant:

In [ ]:
df.to_parquet('./data/Panoma_Field_Permian.parquet.gzip', compression='gzip')

If we are comfortable with SQL, or have an existing database, we may wish to write our dataframe as a table there. We will use the Python implementation of [sqlite](https://www.sqlite.com/index.html), [sqlite3](https://docs.python.org/3/library/sqlite3.html). If you have an existing database you may prefer to look at [SQLalchemy](https://docs.sqlalchemy.org/) to create the connection instead:

In [ ]:
import sqlite3

In [ ]:
connection = sqlite3.connect('./data/panoma.db')
df.to_sql('panoma_raw', con=connection, if_exists='replace', index=False)

You could use `read_sql` to get data from a SQL database instead of reading a file.

<div style="background: #e0f0ff; border: solid 2px #d0e0f0; border-radius:3px; padding: 1em; color: navy">

<h3>High performance Pandas</h3>

Note that for very large datasets, there are a few optional dependencies and settings you can use to speed up certain operations (big data, Boolean comparisons, lots of NaNs, etc).

Read more in the docs, eg:

- https://pandas.pydata.org/docs/user_guide/basics.html#accelerated-operations
- https://pandas.pydata.org/docs/getting_started/install.html#performance-dependencies-recommended
</div>

<hr />

<p style="color:gray">©2022 Agile Geoscience. Licensed CC-BY.</p>